In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%reset -f

import time
import os
import pandas as pd

from parser import parse_input
from Gurobi_IP_Modeler import build_ip_model
from pre_process_constraints import pre_process_constraints
from graph_builder import graph_builder
from aux import remove_dominated_cliques
from aux import solution_map
from Cliques import *


In [ ]:
input_folder = "Inputs"
output_dir="Outputs"
os.makedirs(output_dir, exist_ok=True)
instances = sorted([f for f in os.listdir(input_folder) if f.endswith(".txt")])

summary = []

for instance in instances:
    print(f"\n📦 Processing: {instance}")
    instance_path = os.path.join(input_folder, instance)
    parsed_data = parse_input(instance_path)
    best_A = solution_map["Inputs/"+instance]

    # === 1. Model without clique cuts (and pre-processing disabled) ===

    model_no_cuts,X_no_cuts,Y_no_cuts=build_ip_model(parsed_data,
                                                     best_A,
                                                     clique_constraints=None,
                                                     ip_filename=None)
    model_no_cuts.setParam("PreCrush",1)
    model_no_cuts.setParam("PreSolve",0)
    model_no_cuts.setParam("Cuts",0)
    start=time.time()
    model_no_cuts.optimize()
    time_no_cuts = time.time()-start
    obj_no_cuts = model_no_cuts.objVal
    print(f"🟢 Time without cuts for instance {instance}: {time_no_cuts:.4f}s")


    #=== 2. Cut generation ===

    start=time.time()
    A, b = pre_process_constraints(parsed_data)
    S = clique_detection(A, b)
    G=graph_builder(S)
    S_extended = clique_extension(G,S)
    G_all = graph_builder(S_extended)
    S_strenghtned = remove_dominated_cliques(S_extended)


    model_best,added_cliques,Xcut,Ycut=clique_cut_separator(parsed_data,
                                                            best_A,
                                                            G_all,
                                                            S_strenghtned,
                                                            max_rounds=20,
                                                            min_viol=1e-4,
                                                            instance_name=instance)
    time_cut_gen=time.time()-start
    total_cuts = len(added_cliques)
    print(f"✅ {total_cuts} clique cuts added for {instance}")
    print(f"✂️ Time to generate cuts: {time_cut_gen:.4f}s")
    len_S = len(S)
    len_S_extended = len(S_extended)
    len_S_strenghtned = len(S_strenghtned)

    # === 3. Solve with only violated cuts ===

    model_best_cuts,Xbest,Ybest=build_ip_model(parsed_data,
                                          best_A,
                                          clique_constraints=added_cliques,
                                          ip_filename=None)
    model_best_cuts.setParam("PreCrush",1)
    model_best_cuts.setParam("PreSolve",0)
    model_best_cuts.setParam("Cuts",0)
    start=time.time()
    model_best_cuts.optimize()
    time_best_cuts = time.time()-start
    obj_best_cuts = model_best_cuts.objVal
    print(f"🔵 Time with best cuts: {time_best_cuts:.4f}s")

    # === 5. Summary Entry ===
    summary.append([instance,
                    len_S,
                    len_S_extended,
                    len_S_strenghtned,
                    total_cuts,
                    time_no_cuts,
                    time_cut_gen,
                    time_best_cuts,
                    obj_no_cuts,
                    obj_best_cuts,])

# === Save as DataFrame ===
df=pd.DataFrame(summary,columns=[
    "Instance",
    "Len_S",
    "Len_S_extended",
    "Len_S_strenght",
    "Total cuts",
    "Time_No_Cuts",
    "Time_Cut_Gen",
    "Time_Best_Cuts",
    "Time_All_Cuts",-
    "Object_No_Cuts",
    "Object_Best_Cuts",
    "Object_All_Cuts"])
print("\n✅ Benchmark complete. Results saved.")